# Importing necessary packages

In [1]:
import scala.io.Source
import scala.util.control.Breaks._
import scala.util.matching.Regex 
import org.apache.spark.rdd.RDD
import java.io._
import org.apache.spark.graphx._
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.util.MurmurHash
import org.apache.hadoop.conf._
import org.apache.hadoop.io._
import org.apache.hadoop.mapreduce.lib.input._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.8:4041
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1575541529455)
SparkSession available as 'spark'


import scala.io.Source
import scala.util.control.Breaks._
import scala.util.matching.Regex
import org.apache.spark.rdd.RDD
import java.io._
import org.apache.spark.graphx._
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.util.MurmurHash
import org.apache.hadoop.conf._
import org.apache.hadoop.io._
import org.apache.hadoop.mapreduce.lib.input._


# Taking pair file as a input

In [2]:
val input = sc.textFile("pairs2.txt").map(i=>i.split("\\s+"))
val pairInput = input.map(x=>(x(0),x(1))).distinct().groupByKey().cache()

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at <console>:53
pairInput: org.apache.spark.rdd.RDD[(String, Iterable[String])] = ShuffledRDD[7] at groupByKey at <console>:54


In [3]:
var ranks=pairInput.mapValues(link=>1.0)

ranks: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[8] at mapValues at <console>:54


In [4]:
var D=0.85

D: Double = 0.85


# Calulating page rank 10 iteration

In [5]:
for(i<-1 to 10){
    println("iteration:"+i)
    val contribs=pairInput.join(ranks).values.flatMap{case(papers,rank)=>
    val size=papers.size
        papers.map(paper=>(paper,rank/size))
    }
    ranks=contribs.reduceByKey(_+_).mapValues{
        x=>(1.0-D)+D*x
    }
    ranks.take(10).foreach(println)
    
    
}

iteration:1
(5390b24420f70186a0ee7186,0.38310606060606067)
(5390b56a20f70186a0f04e49,3.890644530848317)
(5390bda020f70186a0f45b58,0.16666666666666669)
(5390a17720f70186a0e522d3,0.4515873015873016)
(55323b9045cec66b6f9da28c,0.36250000000000004)
(55323b1945cec66b6f9d952a,1.085)
(53908bad20f70186a0dc31ab,0.9709575429201688)
(5390b9d520f70186a0f30652,0.2862179487179487)
(5390b7fe20f70186a0f27171,0.21789554531490019)
(5390881220f70186a0d7e2b3,0.48484848484848486)
iteration:2
(5390b24420f70186a0ee7186,0.21170370370370373)
(5390b56a20f70186a0f04e49,0.5718894804492038)
(5390a17720f70186a0e522d3,0.186468253968254)
(55323b1945cec66b6f9d952a,0.17479166666666668)
(53908bad20f70186a0dc31ab,1.5649743797732802)
(5390b9d520f70186a0f30652,0.16273279352226722)
(5390b7fe20f70186a0f27171,0.1809685597149133)
(5390881220f70186a0d7e2b3,0.6732034566539465)
(5390af8920f70186a0ed0c0b,0.22012500000000002)
(5390b48420f70186a0efaa77,0.1555092592592593)
iteration:3
(5390b24420f70186a0ee7186,0.18474252197455326)
(53

# Sorting the data according to rank of a paper

In [7]:
ranks=ranks.sortBy(_._2,false)
val d=ranks.coalesce(1).map(x => x._1).take(10)
val d2=ranks.coalesce(1).map(x => x._2).take(10)
ranks.coalesce(1).saveAsTextFile("rank")

ranks: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[94] at sortBy at <console>:54
d: Array[String] = Array(5390880d20f70186a0d7b7bd, 5390880720f70186a0d7ac77, 539087f820f70186a0d72e6f, 539088b920f70186a0d9141d, 539087eb20f70186a0d6a53d, 539087e720f70186a0d680e0, 53908bad20f70186a0dc2682, 53908bad20f70186a0dc2683, 539088b920f70186a0d9150f, 5390882420f70186a0d88287)
d2: Array[Double] = Array(125.42821239469293, 107.17392881789931, 95.0041467360894, 94.35244283669871, 94.06741599969891, 91.03630958884195, 89.9775517821159, 89.30173919932302, 86.87160212660079, 76.17147135124247)


# Taking rank file as a input

In [8]:
val rank = sc.textFile("rank/part-00000").map(i=>i.split(","))
val rank2 = rank.map(x=>(x(0),x(1)))

rank: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[103] at map at <console>:53
rank2: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[104] at map at <console>:54


In [9]:
@transient val hadoopConf= new Configuration
hadoopConf.set("textinputformat.record.delimiter","#*","#*")

hadoopConf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml


# Input for original data

In [10]:
val path = "citation-acm-v8.txt"

path: String = citation-acm-v8.txt


In [11]:
val inputrdd= sc.newAPIHadoopFile(path,classOf[TextInputFormat],
classOf[LongWritable], classOf[Text], hadoopConf).map{case(key,value)=>"#*"+value.toString}.
filter(value=>value.length!=0)

inputrdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[107] at filter at <console>:59


In [12]:
val a=inputrdd.collect()

a: Array[String] =
Array(#*, "#*INFORMS Journal on Computing
#t2014
#cINFORMS Journal on Computing
#index558ac6e0612c41e6b9d39eed

", "#*Pushout-complements and basic concepts of grammars in toposes
#@Yasuo Kawahara
#t1990
#cTheoretical Computer Science
#index5390879920f70186a0d422b8

", "#*Effective constructors the formal series of trees (French)
#@Symeon Bozapalidis
#t1990
#cTheoretical Computer Science
#index5390879920f70186a0d422b6

", "#*The DataPaper: living in the virtual world
#@Mark Green, Chris Shaw
#t1990
#cGraphics Interface 1990
#index555aa9a345ce207198fe0ae8
#!Virtual reality user interfaces are a new type of user
interface based on placing the user in a three dimensional
environment that can be directly manipulated. Virtual reality
user interfaces have a wide range of ap...

# Filtering out the top ranked paper title and writing it to file

In [13]:
val pw = new PrintWriter(new File("top.txt" ))
var pairs22:Map[String,String]=Map()
for ( e<- d)
    {
        for(d<-a)
        {
            if(d.contains("#index"+e))
            {
                val g = new Regex("#[*](.)*")
                val x=((g findAllIn d).toList)
                x.foreach(n=>pw.write(n.substring(2)))
            }
        }
                   pw.write(","+e+"\n")       
    }
pw.close

pw: java.io.PrintWriter = java.io.PrintWriter@507d9bc7
pairs22: Map[String,String] = Map()


# Taking pair file as a input

In [14]:
val input = sc.textFile("pairs2.txt").map(i=>i.split("\\s+"))
val pairInput = input.map(x=>(x(0),x(1)))

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[110] at map at <console>:55
pairInput: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[111] at map at <console>:56


# Hashing the paper id

In [15]:
    val zipPair = pairInput.map(_._1).distinct.zipWithIndex.collect.toMap
    val zipPair2 = pairInput.map(_._2).distinct.zipWithIndex.collect.toMap
    val pairInputHash = pairInput.map{case(v1,v2)=>(zipPair.get(v1),zipPair2.get(v2))}
    val cleanInput = pairInputHash.map{case(Some(v1),Some(v2))=>(v1,v2)}

<console>:57: warning: match may not be exhaustive.
It would fail on the following inputs: (None, None), (None, Some(_)), (Some(_), None)
           val cleanInput = pairInputHash.map{case(Some(v1),Some(v2))=>(v1,v2)}
                                             ^
zipPair: scala.collection.immutable.Map[String,Long] = Map(5390980720f70186a0e02cb9 -> 402735, 5390b5df20f70186a0f0b7da -> 105409, 5390aaf920f70186a0eaefb7 -> 294191, 53908b2a20f70186a0db8f14 -> 670846, 5390a77d20f70186a0e8e8c9 -> 576295, 53909f6920f70186a0e3a197 -> 508660, 5390aaf920f70186a0eaeca1 -> 686090, 559253ae0cf28b1a968ffc09 -> 702136, 5390b3da20f70186a0ef5e07 -> 791700, 558b3498612c41e6b9d4697e -> 898659, 5390b4c420f70186a0eff128 -> 621443, 5390a0b720f70186a0e4e4fd -> 102400, 5390a0b720f70186a0e50068 -> 6035, 53908b4920f70186a0dbcb40 -> 725862, 53908adf20f70186a0daca84 -> 317986, 53909f2d20f70186a0e3920b -> 869840, 5390bf1320f70186a0f510c3 -> 334928, 5390981d20f70186a0e05979 -> 476586, 53908b1820f70186a0db39ba -> 20

# Writing hashing pair to file

In [16]:
val pw = new PrintWriter(new File("sss.txt" ))
zipPair2.foreach(x=>pw.write((x.toString.substring(1,x.toString.length()-1)+"\n")))
pw.close()  

pw: java.io.PrintWriter = java.io.PrintWriter@10a124e7


In [17]:
@transient val hadoopConf= new Configuration
hadoopConf.set("textinputformat.record.delimiter","\n")

hadoopConf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml


In [18]:
val x= "sss.txt"

x: String = sss.txt


In [19]:
val inputrdd2= sc.newAPIHadoopFile(x,classOf[TextInputFormat],
classOf[LongWritable], classOf[Text], hadoopConf).map{case(key,value)=>value.toString}.
filter(value=>value.length!=0)

inputrdd2: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[126] at filter at <console>:59


# Converting hash pair to arrays

In [20]:
val z=inputrdd2.collect()

z: Array[String] = Array(5390b1d220f70186a0ee1715,823874, 5390b5df20f70186a0f0b7da,115996, 5390b5c720f70186a0f0951e,335267, 5390aaf920f70186a0eaefb7,323509, 5390975920f70186a0dfc69d,58031, 53908b2a20f70186a0db8f14,738281, 5390995d20f70186a0e156cc,715583, 539095ba20f70186a0df0b83,531518, 539087f320f70186a0d6fbd7,594293, 5390975920f70186a0dfe2ff,58870, 5390b20120f70186a0ee3e4b,885001, 53909f6920f70186a0e3a197,559753, 5390aaf920f70186a0eaeca1,755149, 5390881220f70186a0d7e6d6,715043, 5390b3da20f70186a0ef5e07,871620, 53908aac20f70186a0da75de,214536, 539087c320f70186a0d55abc,285192, 5390b4c420f70186a0eff128,683868, 5390a37f20f70186a0e6d05e,84691, 5390a0b720f70186a0e4e4fd,112668, 5390a0b720f70186a0e50068,6665, 53908adf20f70186a0daca84,349428, 53909f2d20f70186a0e3920b,957365, 5390981d20f70186a0...

# Filtering out top 10 hash pair

In [21]:
val pw = new PrintWriter(new File("top2.txt" ))
var pairs22:Map[String,String]=Map()
for ( e<- d)
    {
        for(d<-z)
        {
            //println(d)
            if(d.contains(e))
            {
               
                pw.write(d+"\n")
                
            }
        }
                 //  pw.write(","+e+"\n")       
    }
pw.close

pw: java.io.PrintWriter = java.io.PrintWriter@7034e1e2
pairs22: Map[String,String] = Map()


# making graph

In [22]:
    val edges: RDD[(VertexId,VertexId)] = cleanInput
    val graph = Graph.fromEdgeTuples(edges,null)
    

edges: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId)] = MapPartitionsRDD[123] at map at <console>:57
graph: org.apache.spark.graphx.Graph[Null,Int] = org.apache.spark.graphx.impl.GraphImpl@1f7fd3a8


In [23]:
val inlinks = graph.inDegrees
val outlinks = graph.outDegrees
    

inlinks: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[140] at RDD at VertexRDD.scala:57
outlinks: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[144] at RDD at VertexRDD.scala:57


# making inlinks as array

In [24]:
val in=inlinks.collect()

in: Array[(org.apache.spark.graphx.VertexId, Int)] = Array((682981,20), (354796,7), (755508,4), (194402,2), (754897,4), (846729,206), (101257,1), (165737,5), (934765,6), (619983,1), (735956,1), (648817,1), (814762,1), (447187,14), (985335,1), (237445,38), (903110,7), (857454,2), (904930,1), (720200,4), (161980,3), (980434,7), (363818,49), (422955,2), (308958,13), (162877,2), (581139,113), (547183,1), (4992,3), (571636,1), (853554,2), (585442,10), (260793,11), (496158,18), (813189,1), (941213,1), (411294,2), (116662,1), (851916,1), (442260,2), (551291,18), (928616,11), (245388,6), (192192,6), (584766,1), (316030,4), (61139,17), (9867,1), (355173,2), (683020,4), (519922,5), (167635,1), (272285,2), (992030,2), (748293,1), (559624,3), (774852,3), (459277,3), (451607,5), (969423,2), (299897,...

# writing inlinks to file

In [25]:
val pw = new PrintWriter(new File("inlinks.txt" ))
in.foreach(x=>pw.write((x.toString.substring(1,x.toString.length()-1)+"\n")))
pw.close

pw: java.io.PrintWriter = java.io.PrintWriter@170c8a7b


# Taking inlinks file as input

In [26]:
@transient val hadoopConf= new Configuration
hadoopConf.set("textinputformat.record.delimiter","\n")
val x= "inlinks.txt"
val inputrdd3= sc.newAPIHadoopFile(x,classOf[TextInputFormat],
classOf[LongWritable], classOf[Text], hadoopConf).map{case(key,value)=>value.toString}.
filter(value=>value.length!=0)

hadoopConf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
x: String = inlinks.txt
inputrdd3: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[147] at filter at <console>:62


In [27]:
inputrdd3.foreach(println)

682981,20
354796,7
755508,4
194402,2
754897,4
846729,206
101257,1
165737,5
934765,6
619983,1
735956,1
648817,1
814762,1
447187,14
985335,1
237445,38
903110,7
857454,2
904930,1
720200,4
161980,3
980434,7
363818,49
422955,2
308958,13
162877,2
581139,113
547183,1
4992,3
571636,1
853554,2
585442,10
260793,11
496158,18
813189,1
941213,1
411294,2
116662,1
851916,1
442260,2
551291,18
928616,11
245388,6
192192,6
584766,1
316030,4
61139,17
9867,1
355173,2
683020,4
519922,5
167635,1
272285,2
992030,2
748293,1
559624,3
774852,3
459277,3
451607,5
969423,2
299897,13
511472,2
591825,4
754715,5
2938,3
125957,1
210106,1
57616,10
245219,2
968006,2
215410,2
922298,2
777790,34
388544,4
708877,3
806923,2
739492,10
327457,14
638300,3
939016,25
725530,5
34931,8
202995,5
526513,10
956644,12
548262,1
98761,11
889954,9
734669,13
415844,5
190424,1
259311,1
742430,9
86970,1
314379,4
331968,14
742729,15
750204,1
891553,1
409227,14
460213,3
78416,13
100802,6
465803,7
894322,1
833547,2
75790,29
608933,1
317967,2
76

419016,1
591123,1
17225,11
675909,10
424372,8
503802,2
554125,1
175942,1
755820,4
316836,1
19370,2
790738,1
53534,1
77311,1
333164,1
193284,1
242632,1
837356,2
441584,4
302757,56
276627,1
383552,1
209443,27
128765,21
977834,1
165360,2
80301,3
946452,6
930709,5
614731,22
735358,1
583804,15
70265,1
309868,3
462228,5
285298,5
972361,2
767065,3
721890,11
187889,16
203788,7
998062,3
216814,4
704691,2
12337,11
504374,1
133835,51
861614,1
442650,9
424203,11
112879,2
197548,1
947895,1
723359,4
57551,13
558701,5
380419,2
67015,3
265954,8
235391,2
920738,4
83057,2
314210,2
339924,2
127348,2
575328,12
681967,2
111566,1
846521,27
900536,2
286598,8
863434,1
113867,4
867594,4
439439,3
662766,1
966017,1
441610,2
242086,5
753454,7
676520,4
404105,1
582946,2
209144,1
129532,6
239629,9
355264,1
766207,1
465491,1
307658,6
319943,4
640341,5
926159,1
776139,3
494494,1
95888,3
964821,1
895544,3
650013,7
922181,19
134212,5
451295,3
441701,1
504621,2
780364,1
556283,5
617539,4
418288,6
815516,2
801632,1
80382

799812,1
795080,7
777400,2
414583,7
875654,5
588653,5
984932,7
967863,1
841568,2
469118,2
284531,5
256269,3
887497,46
459524,11
314561,3
606840,16
625287,1
612976,5
700466,1
637273,23
979836,5
123825,1
161642,1
959504,1
552097,1
599690,10
962676,2
622713,6
864942,7
968279,20
531544,5
896857,1
579410,1
213278,1
543985,26
479518,10
713882,1
10881,8
216229,1
715260,3
400166,12
960674,15
90402,4
3289,1
290524,32
393575,1
697957,2
435422,1
993889,8
975182,8
918567,1
990886,7
532896,1
622141,4
464529,7
225069,1
558779,31
817336,3
101881,1
349843,77
605293,1
521170,1
49036,6
33332,1
309036,24
148850,1
527540,1
587379,2
419211,1
557479,2
43706,1
125398,13
179075,1
364806,5
517348,1
739882,2
479934,33
107107,5
151437,1
538590,9
349089,2
79664,4
725166,2
23738,1
611195,1
293423,34
104546,4
970047,4
444522,2
497874,16
587548,1
942838,1
134316,3
309075,1
518700,5
585052,1
384540,1
947973,1
7137,2
642967,2
718081,10
24453,2
751413,1
420472,20
55588,17
131196,7
359099,1
17693,3
976976,4
437229,5
862

353509,1
309855,12
755846,1
622856,1
754637,1
218907,4
385099,3
611221,9
428766,1
515125,6
322322,2
611572,44
685087,6
785603,1
796939,2
888563,33
69732,1
124384,1
403507,6
960362,4
922909,15
752349,12
694395,1
431834,3
642772,1
372177,4
25051,3
326950,7
610805,2
352586,1
739791,2
354042,1
111501,3
754377,6
93366,4
752622,2
911976,2
526695,90
193245,1
767481,2
788411,1
412893,12
151138,9
364351,1
833287,1
840697,1
515476,5
135746,27
51974,1
614913,3
758797,2
724503,7
391560,7
331253,1
734474,1
694980,1
54145,1
21775,9
250250,1
228579,2
238485,11
341497,1
324311,4
215202,17
219700,1
912132,1
63362,8
532844,70
595920,8
26715,1
758446,4
170677,1
964652,1
558922,2
126555,5
458068,1
480012,3
405015,16
249730,5
597714,1
623246,8
540592,4
712920,1
363909,1
603486,1
266968,14
632996,2
191516,1
874003,3
611715,9
95901,2
173862,4
232037,3
629213,2
644943,1
792597,9
639470,2
333476,1
960323,1
196417,6
699140,3
144521,3
998855,3
466375,21
390273,12
442403,1
626288,1
759018,4
593164,4
544284,6
9643

729469,38
132951,4
876941,17
402207,5
631501,6
674986,10
295503,3
575770,2
327314,3
316069,4
66170,7
808561,4
277303,1
251004,1
497497,1
881504,2
790855,6
434395,1
330538,2
26585,1
704613,5
406367,3
745446,1
970710,2
817219,20
56225,10
903175,1
319696,34
897832,4
793013,12
274924,2
292812,7
635765,9
954850,3
42822,5
10764,3
655096,4
416520,1
762255,1
355875,1
879775,13
916942,2
241592,5
77272,4
292227,23
93756,2
440505,2
989586,1
618020,2
915395,27
212680,2
190619,3
523757,3
355342,1
275223,2
759876,2
385840,18
722163,3
841178,16
884650,1
585065,21
934323,1
640081,27
407381,6
549744,1
843934,2
388011,1
243347,3
975325,14
666172,4
489411,6
691548,20
240591,8
77870,1
844506,3
384033,1
520156,5
578669,48
536913,1
593073,1
370058,1
239213,2
587041,3
30186,16
433706,8
163592,8
715624,1
330772,17
455039,4
709072,1
498784,1
591227,64
46618,6
387517,1
369122,1
240734,4
729339,1
246428,1
692237,9
311714,1
928525,2
658775,1
431990,10
98566,2
763503,2
73931,3
64714,9
32877,6
26377,2
764725,2
3322

...

585805,5
345370,5
609647,53
863862,3
973686,1
785407,10
177982,2
105494,1
674868,9
519141,1
851590,8
335737,2
486459,2
267331,6
535586,7
400542,3
825928,1
224808,37
447225,3
971723,1
804205,4
357564,1
336998,3
156259,2
278550,5
849757,1
315886,2
349023,1
994044,1
26168,3
688583,6
776970,1
28404,9
183663,1
234012,21
152164,2
2209,1
673438,1
891136,1
631890,4
943006,1
625806,1
899313,5
103388,1
990391,2
685762,12
866358,1
408264,16
118052,6
442454,1
448954,1
403493,5
488656,1
667601,1
590472,3
948804,50
148784,5
583296,1
653535,2
302379,1
68405,2
878500,4
587924,2
772537,6
587261,1
521195,4
444274,6
228396,1
626495,17
1005367,1
321333,3
303835,130
770002,25
117116,14
660828,4
737710,5
205256,3
587651,13
617109,1
619397,1
221064,4
275287,4
530672,1
969006,9
663337,3
548924,1
318603,1
536730,5
635985,16
208792,5
272947,2
63374,5
532856,5
749709,4
615653,6
864161,6
166516,1
202383,2
174290,1
477190,5
909713,1
871493,7
378533,1
12466,1
419782,3
682434,7
677546,1
981538,2
604980,11
296971,74


53468,15
212861,2
295229,13
458587,15
1005601,1
912105,3
854944,19
628042,1
475513,1
599481,4
19330,2
997619,3
843907,2
571570,371
876927,2
827878,1
617551,9
522183,6
221922,3
934998,20
543919,12
25804,3
344850,21
754077,3
434745,137
417078,2
863108,2
194999,1
839708,1
158196,1
64570,1
833598,67
99384,10
755143,7
35047,1
912586,21
956617,4
588327,35
361594,6
905163,36
541904,9
18719,1
351480,3
514955,7
493453,1
597557,1
522664,4
375816,9
657110,2
304329,1
767441,1
200394,3
903772,5
9307,2
328418,25
630200,17
169779,31
363050,16
692106,16
751841,5
99865,9
533246,2
530074,59
228942,1
396863,3
789151,12
556685,4
530555,6
154166,14
421706,2
213914,57
779739,16
167933,1
818557,12
250782,1
96563,2
525641,7
270035,1
735955,2
1006238,1
285843,13
469468,1
636609,1
57069,13
861314,1
973959,1
516320,10
278199,1
523028,3
535820,5
755624,14
743729,2
823016,1
834443,7
217437,26
253941,1
449877,2
529749,1
550263,3
980004,2
424553,2
647529,1
448837,7
144338,3
639027,1
72006,6
4627,3
707082,4
255358,3


524289,1
257347,2
941485,21
951157,1
349036,1
44264,21
327014,1
983839,8
199627,1
251081,7
900028,8
776229,10
335113,4
475474,36
696383,5
305083,5
585012,6
723397,13
585324,1
546454,3
802450,3
348113,1
474577,1
312610,2
131416,2
30822,3
579201,8
586182,1
824719,15
916915,3
360697,1
72409,1
610011,1
46682,1
63634,3
308788,1
979185,2
875302,5
480128,5
907204,2
934738,2
990521,34
981057,3
898767,2
566474,1
374542,15
797250,1
248026,2
35554,1
432990,2
576367,30
768533,15
198457,2
849510,4
888224,5
223898,1
78922,2
15989,1
463891,15
280864,1
299844,1
57290,2
824433,6
502943,7
1003742,1
482702,2
339585,3
387269,2
178788,4
151553,1
553253,1
502566,1
995929,1
111578,11
766869,18
244581,5
194986,1
705743,8
994421,1
9346,4
757158,1
223950,1
140464,4
679353,3
613742,8
905657,13
608750,7
734850,1
2885,4
908738,9
342731,1
391572,2
963962,3
191645,4
52402,2
444482,3
318993,4
116310,26
80131,1
313221,1
1006693,1
109849,2
924689,3
212679,1
949441,3
15586,1
556295,1
763281,7
407497,1
684358,1
173783,10

376973,1
962649,19
294943,1
733537,12
192308,6
774162,5
316692,6
355952,9
134081,2
851681,4
798277,2
633359,1
520363,5
613014,1
153438,1
792791,1
392690,5
998841,7
273285,3
580579,4
199913,1
838785,3
882686,83
754935,2
112449,2
713569,1
802593,3
222247,1
539096,37
811394,1
560650,1
150942,1
133574,3
509456,5
628107,4
425541,8
522092,2
476566,1
191554,6
857492,6
404910,1
655953,1
793415,1
67859,1
333111,1
818440,25
566409,1
974427,11
1000921,12
950234,1
698762,8
374919,1
535001,8
374035,3
940900,6
184664,13
795261,61
182454,5
440543,2
823497,1
422824,1
494246,8
508182,76
508260,5
629095,1
599676,1
411033,11
755520,8
895582,3
931514,4
619540,3
146951,1
400438,1
419197,2
643447,7
183468,1
60423,4
858597,1
954511,10
311427,38
452087,20
517776,1
448421,5
409252,2
759329,6
898611,87
497509,2
901965,1
289522,35
100918,17
378676,2
856504,5
991561,1
156402,6
318837,13
790633,38
434056,2
485965,1
851941,2
443507,8
224743,5
103024,5
934790,4
361997,2
42067,1
389622,1
930682,8
27130,1
281020,1
915

561573,4
452594,4
117584,5
611233,50
41170,2
422616,4
418196,4
157429,3
723644,1
147081,24
580852,7
467570,1
34592,5
232998,1
411618,5
452035,7
82640,1
883648,6
342484,12
912313,1
10165,4
364116,9
145625,2
26623,1
721551,3
696240,16
746303,2
989013,1
456039,5
993563,1
151774,12
748318,3
453023,2
563939,4
222078,9
270763,2
834235,1
1005289,1
553682,1
59227,5
189981,4
772966,2
720758,3
692288,2
262287,1
910233,1
525095,7
707862,1


# taking top 10 hash pair

In [28]:
val input = sc.textFile("top2.txt").map(i=>i.split(","))
val topinl = input.map(x=>(x(1)))

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[150] at map at <console>:57
topinl: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[151] at map at <console>:58


In [29]:
topinl.foreach(println)

943811
549060
117572
656042
590378
238470
553422
131528
855727
604077


In [30]:
val inlk=inputrdd3.collect()

inlk: Array[String] = Array(682981,20, 354796,7, 755508,4, 194402,2, 754897,4, 846729,206, 101257,1, 165737,5, 934765,6, 619983,1, 735956,1, 648817,1, 814762,1, 447187,14, 985335,1, 237445,38, 903110,7, 857454,2, 904930,1, 720200,4, 161980,3, 980434,7, 363818,49, 422955,2, 308958,13, 162877,2, 581139,113, 547183,1, 4992,3, 571636,1, 853554,2, 585442,10, 260793,11, 496158,18, 813189,1, 941213,1, 411294,2, 116662,1, 851916,1, 442260,2, 551291,18, 928616,11, 245388,6, 192192,6, 584766,1, 316030,4, 61139,17, 9867,1, 355173,2, 683020,4, 519922,5, 167635,1, 272285,2, 992030,2, 748293,1, 559624,3, 774852,3, 459277,3, 451607,5, 969423,2, 299897,13, 511472,2, 591825,4, 754715,5, 2938,3, 125957,1, 210106,1, 57616,10, 245219,2, 968006,2, 215410,2, 922298,2, 777790,34, 388544,4, 708877,3, 806923,2,...

In [31]:
val t=topinl.collect()

t: Array[String] = Array(943811, 117572, 590378, 553422, 855727, 604077, 549060, 656042, 238470, 131528)


# Filtering out the ranks of top 10 and writing it file

In [32]:
val pw = new PrintWriter(new File("top3.txt" ))
var pairs22:Map[String,String]=Map()
for ( e<- t)
    {
        for(d<-inlk)
        {
            if(d.contains(e))
            pw.write(d+"\n")
           
        }
                 //  pw.write(","+e+"\n")       
    }
pw.close

pw: java.io.PrintWriter = java.io.PrintWriter@3f67f035
pairs22: Map[String,String] = Map()


# taking top 10 rank as input from file 

In [33]:
val input = sc.textFile("top.txt").map(i=>i.split(","))
val top = input.map(x=>(x(0))).take(10)

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[154] at map at <console>:57
top: Array[String] = Array(A relational model of data for large shared data banks, A method for obtaining digital signatures and public-key cryptosystems, Database abstractions: aggregation and generalization, Computers and Intractability: A Guide to the Theory of NP-Completeness, The art of computer programming, The art of computer programming, Recovery semantics for a DB/DC system, Recovery scenario for a DB/DC system, The Design and Analysis of Computer Algorithms, Theory of Syntactic Recognition for Natural Languages)


In [34]:
val toprank=ranks.map(x => x._2).take(10)

toprank: Array[Double] = Array(125.42821239469293, 107.17392881789931, 95.0041467360894, 94.35244283669871, 94.06741599969891, 91.03630958884195, 89.9775517821159, 89.30173919932302, 86.87160212660079, 76.17147135124247)


In [35]:
val input = sc.textFile("top3.txt").map(i=>i.split(","))
val topinl = input.map(x=>(x(1))).take(10)

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[159] at map at <console>:57
topinl: Array[String] = Array(1526, 2223, 366, 8619, 1640, 1261, 33, 16, 2024, 190)


# Printing the top 10 Paper with rank and inlinks

In [41]:
val pw = new PrintWriter(new File("paper_rank.txt" ))
println("Paper-title"+"\t"+"Number_of_in-liks"+"\t"+"page_rank")
pw.write("Paper-title"+"\t"+"Number_of_in-liks"+"\t"+"page_rank"+"\n")
for(i<-0 to 9)
{
    println(top(i)+"\t"+topinl(i)+"\t"+toprank(i))
    pw.write(top(i)+"\t"+topinl(i)+"\t"+toprank(i)+"\n")
}
pw.close

Paper-title	Number_of_in-liks	page_rank
A relational model of data for large shared data banks	1526	125.42821239469293
A method for obtaining digital signatures and public-key cryptosystems	2223	107.17392881789931
Database abstractions: aggregation and generalization	366	95.0041467360894
Computers and Intractability: A Guide to the Theory of NP-Completeness	8619	94.35244283669871
The art of computer programming	1640	94.06741599969891
The art of computer programming	1261	91.03630958884195
Recovery semantics for a DB/DC system	33	89.9775517821159
Recovery scenario for a DB/DC system	16	89.30173919932302
The Design and Analysis of Computer Algorithms	2024	86.87160212660079
Theory of Syntactic Recognition for Natural Languages	190	76.17147135124247


pw: java.io.PrintWriter = java.io.PrintWriter@3c60343b
